In [9]:
import numpy as np
import cv2
import dlib
from imutils import face_utils

## Perform facial landmark Detection

In [11]:
# Read Image
im = cv2.imread("test.jpg");
size = im.shape
print(size)
# Directory of the pretrained model
p = "shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Get faces into webcam's image
rects = detector(im, 0)

# Loop over every face detected. Here we only have on face each time
for rect in rects:
    # Make the prediction and transfom it to numpy array
    shape = predictor(im, rect)
    landmarks = face_utils.shape_to_np(shape)

# Grab the 2D coordinates of our six sample points
image_points = np.array([
                            landmarks[33, :],     # Nose tip
                            landmarks[8, :],     # Chin
                            landmarks[36, :],     # Left eye left corner
                            landmarks[45, :],     # Right eye right corner
                            landmarks[48, :],     # Left Mouth corner
                            landmarks[54, :]      # Right mouth corner
                        ], dtype="double")

# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                         
                        ])

# Approximate camera intrinsic parameters
focal_length = size[1]
center = (size[1]/2, size[0]/2)
camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )
dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
 
print("Camera Matrix :\n {0}".format(camera_matrix))

# Solve the PnP problem with the parameters specified above
(success, rotation_vector, translation_vector) = cv2.solvePnP(
    model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
    )

print("Rotation Vector:\n {0}".format(rotation_vector))
print("Translation Vector:\n {0}".format(translation_vector))

# Project a 3D point (0, 0, 1000.0) onto the image plane.
# We use this to draw a line sticking out of the nose
(nose_end_point2D, jacobian) = cv2.projectPoints(
    np.array([(0.0, 0.0, 500.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
    )

for p in image_points:
    cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
    
p1 = ( int(image_points[0][0]), int(image_points[0][1]) )
p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )

# Draw a line connecting the two points. This line must show
# the direction out of the nose
cv2.line(im, p1, p2, (255,0,0), 2)
 
# Display image
cv2.imshow("Output", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

(480, 640, 3)
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.82303234]
 [ 0.07662005]
 [ 0.58571914]]
Translation Vector:
 [[ -29.86482998]
 [ 125.23345586]
 [1667.60154487]]


In [18]:
# In the beggining we will use the rotation and translation of each face as input
X = np.concatenate((rotation_vector, translation_vector), axis=1)
print(X.shape)

# The selected image is a 'looking' image
y = np.ones(1)
print(y.shape)

(3, 2)
(1,)
